In [6]:
from transformers import TFXLMRobertaForSequenceClassification, XLMRobertaTokenizer
import tensorflow as tf

model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
model = TFXLMRobertaForSequenceClassification.from_pretrained(model_path, from_pt=True)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)



/home/apprenant/miniconda3/envs/TF_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-22 14:32:01.436088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 14:32:01.436942: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This

In [4]:
export_path = "exported_model/hugging_model"
model.save_pretrained(export_path, saved_model=True)

INFO:tensorflow:Assets written to: exported_model/hugging_model/saved_model/1/assets


INFO:tensorflow:Assets written to: exported_model/hugging_model/saved_model/1/assets


In [1]:
import json
import requests
from tensorflow.python.ops.numpy_ops import np_config
import numpy as np

np_config.enable_numpy_behavior()


def classify_text(text):
    url = "http://localhost:8501/v1/models/hugging_model:predict"
    input = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=512)
    instances = [{"input_ids": input_ids.tolist(), "attention_mask": attention_mask.tolist()} for input_ids, attention_mask in zip(input["input_ids"], input["attention_mask"])]
    data = json.dumps({"signature_name": "serving_default", "instances": instances})
    response = requests.post(url, data=data)
    
    raw_predictions = json.loads(response.text)["predictions"]

        # Process the raw predictions to get the desired output format
    labels = ["Negative", "Neutral", "Positive"]
    scores = np.exp(raw_predictions) / np.exp(raw_predictions).sum(-1, keepdims=True)
    top_label_index = np.argmax(scores, axis=-1)[0]
    top_label = labels[top_label_index]
    top_score = scores[0][top_label_index]
    
    result = [{"label": top_label, "score": top_score}]
    return result
    
# tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=hugging_model --model_base_path=$(realpath exported_model/hugging_model/saved_model)


2023-03-22 14:29:51.203144: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
text = "I love this product!"
predictions = classify_text(text)
print(predictions)

ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/hugging_model:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0d48295ea0>: Failed to establish a new connection: [Errno 111] Connection refused'))